# How to use ChatGPT API to build a chatbot for product recommendations with embeddings

**Are you looking to build a chatbot that can recommend products to your customers based on their unique profile?**

Here's a step-by-step guide that shows you how to build a chatbot using embeddings to match a user's profile with relevant products from a company's database.

Detailed step-by-step intructions for this repo in this blog post: https://norahsakal.com/blog/chatgpt-product-recommendation-embeddings

### Important
If you already have openai installed, make sure to call `pip install openai --upgrade` in your terminal to make sure you have access to version `0.27.0` (or higher) which has the newly added **gpt-3.5-turbo model**.

---

## Difference between ChatGPT and GPT-3 API
ChatGPT is OpenAI's new model family designed specifically for chat-based interactions.

Unlike the larger GPT-3 model, ChatGPT can consume a sequence of messages with metadata, which allows for more contextual understanding of conversations.

Plus, the ChatGPT API is currently priced at only $0.002 per 1k tokens, which is 10x cheaper than the existing GPT-3.5 models.

So if you're looking to build a chatbot for support requests, ChatGPT is definitely worth considering.

In [1]:
import openai
import pandas as pd
from openai.embeddings_utils import get_embedding,cosine_similarity


# 1. Add your API key

In [2]:
openai.api_type = "azure"
openai.api_key = "c4a1d51a3b974ac6b0ac45b3db2cd7b4"
openai.api_base = "https://cheppy.openai.azure.com/"
openai.api_version = "2023-03-15-preview"

# 2. Create product data

In [3]:

product_data = [{
    "prod_id": 1,
    "prod": "moisturizer",
    "brand":"Aveeno",
    "description": "for dry skin"
},
{
    "prod_id": 2,
    "prod": "foundation",
    "brand":"Maybelline",
    "description": "medium coverage"
},
{
    "prod_id": 3,
    "prod": "moisturizer",
    "brand":"CeraVe",
    "description": "for dry skin"
},
{
    "prod_id": 4,
    "prod": "nail polish",
    "brand":"OPI",
    "description": "raspberry red"
},
{
    "prod_id": 5,
    "prod": "concealer",
    "brand":"chanel",
    "description": "medium coverage"
},
{
    "prod_id": 6,
    "prod": "moisturizer",
    "brand":"Ole Henkrisen",
    "description": "for oily skin"
},
{
    "prod_id": 7,
    "prod": "moisturizer",
    "brand":"CeraVe",
    "description": "for normal to dry skin"
},
{
    "prod_id": 8,
    "prod": "moisturizer",
    "brand":"First Aid Beauty",
    "description": "for dry skin"
},{
    "prod_id": 9,
    "prod": "makeup sponge",
    "brand":"Sephora",
    "description": "super-soft, exclusive, latex-free foam"
}]

# 3. Add product data to dataframe

In [4]:
product_data_df = pd.DataFrame(product_data)
product_data_df

,prod_id,prod,brand,description
0,1,moisturizer,Aveeno,for dry skin
1,2,foundation,Maybelline,medium coverage
2,3,moisturizer,CeraVe,for dry skin
3,4,nail polish,OPI,raspberry red
4,5,concealer,chanel,medium coverage
5,6,moisturizer,Ole Henkrisen,for oily skin
6,7,moisturizer,CeraVe,for normal to dry skin
7,8,moisturizer,First Aid Beauty,for dry skin
8,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam"


# 4. Create column with combined data

In [5]:
product_data_df['combined'] = product_data_df.apply(lambda row: f"{row['brand']}, {row['prod']}, {row['description']}", axis=1)
product_data_df

,prod_id,prod,brand,description,combined
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin"
1,2,foundation,Maybelline,medium coverage,"Maybelline, foundation, medium coverage"
2,3,moisturizer,CeraVe,for dry skin,"CeraVe, moisturizer, for dry skin"
3,4,nail polish,OPI,raspberry red,"OPI, nail polish, raspberry red"
4,5,concealer,chanel,medium coverage,"chanel, concealer, medium coverage"
5,6,moisturizer,Ole Henkrisen,for oily skin,"Ole Henkrisen, moisturizer, for oily skin"
6,7,moisturizer,CeraVe,for normal to dry skin,"CeraVe, moisturizer, for normal to dry skin"
7,8,moisturizer,First Aid Beauty,for dry skin,"First Aid Beauty, moisturizer, for dry skin"
8,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam","Sephora, makeup sponge, super-soft, exclusive,..."


# 5. Create embeddings for combined product data

In [6]:
product_data_df['text_embedding'] = product_data_df.combined.apply(lambda x: get_embedding(x, engine='text-embedding-ada-002'))
product_data_df

,prod_id,prod,brand,description,combined,text_embedding
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin","[-0.0054472219198942184, -0.009129984304308891..."
1,2,foundation,Maybelline,medium coverage,"Maybelline, foundation, medium coverage","[-0.015995968133211136, 0.002405167557299137, ..."
2,3,moisturizer,CeraVe,for dry skin,"CeraVe, moisturizer, for dry skin","[0.007487323135137558, -0.016988780349493027, ..."
3,4,nail polish,OPI,raspberry red,"OPI, nail polish, raspberry red","[-0.0005623639444820583, -0.01371096819639206,..."
4,5,concealer,chanel,medium coverage,"chanel, concealer, medium coverage","[0.00467997882515192, 0.004558207001537085, 0...."
5,6,moisturizer,Ole Henkrisen,for oily skin,"Ole Henkrisen, moisturizer, for oily skin","[-0.004895348101854324, -0.022356607019901276,..."
6,7,moisturizer,CeraVe,for normal to dry skin,"CeraVe, moisturizer, for normal to dry skin","[0.015787700191140175, -0.01325217541307211, 0..."
7,8,moisturizer,First Aid Beauty,for dry skin,"First Aid Beauty, moisturizer, for dry skin","[-0.011300002224743366, -0.007518302649259567,..."
8,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam","Sephora, makeup sponge, super-soft, exclusive,...","[0.006278818007558584, 0.004901116248220205, 0..."


# 6. Create customer profile data

# 7. Add customer profile data to dataframe

# 8. Create combined column for customer profile

# 9. Create customer profile data embeddings

# 10. Create embeddings for input question

In [81]:
customer_input = "Hi! Can you recommend a good moisturizer for me?"

In [82]:
response = openai.Embedding.create(
    input=customer_input,
    engine="text-embedding-ada-002"
)
embeddings_customer_question = response['data'][0]['embedding']

# 11. Get similarities for purchase history

# 12. Save top 3 similarities for purchase history

# 13. Get similarities for products

In [83]:
product_data_df['search_products'] = product_data_df.text_embedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
product_data_df = product_data_df.sort_values('search_products', ascending=False)
product_data_df


,prod_id,prod,brand,description,combined,text_embedding,search_products
2,3,moisturizer,CeraVe,for dry skin,"CeraVe, moisturizer, for dry skin","[0.007487323135137558, -0.016988780349493027, ...",0.861155
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin","[-0.0054472219198942184, -0.009129984304308891...",0.861080
7,8,moisturizer,First Aid Beauty,for dry skin,"First Aid Beauty, moisturizer, for dry skin","[-0.011300002224743366, -0.007518302649259567,...",0.855234
6,7,moisturizer,CeraVe,for normal to dry skin,"CeraVe, moisturizer, for normal to dry skin","[0.015787700191140175, -0.01325217541307211, 0...",0.851172
5,6,moisturizer,Ole Henkrisen,for oily skin,"Ole Henkrisen, moisturizer, for oily skin","[-0.004895348101854324, -0.022356607019901276,...",0.837502
4,5,concealer,chanel,medium coverage,"chanel, concealer, medium coverage","[0.00467997882515192, 0.004558207001537085, 0....",0.784108
1,2,foundation,Maybelline,medium coverage,"Maybelline, foundation, medium coverage","[-0.015995968133211136, 0.002405167557299137, ...",0.782689
8,9,makeup sponge,Sephora,"super-soft, exclusive, latex-free foam","Sephora, makeup sponge, super-soft, exclusive,...","[0.006278818007558584, 0.004901116248220205, 0...",0.762134
3,4,nail polish,OPI,raspberry red,"OPI, nail polish, raspberry red","[-0.0005623639444820583, -0.01371096819639206,...",0.748637


# 14. Save top 3 similarities for products

In [84]:
top_3_products_df = product_data_df.head(3)
top_3_products_df

,prod_id,prod,brand,description,combined,text_embedding,search_products
2,3,moisturizer,CeraVe,for dry skin,"CeraVe, moisturizer, for dry skin","[0.007487323135137558, -0.016988780349493027, ...",0.861155
0,1,moisturizer,Aveeno,for dry skin,"Aveeno, moisturizer, for dry skin","[-0.0054472219198942184, -0.009129984304308891...",0.861080
7,8,moisturizer,First Aid Beauty,for dry skin,"First Aid Beauty, moisturizer, for dry skin","[-0.011300002224743366, -0.007518302649259567,...",0.855234


# 15. Create prompt

The system message helps set the behavior of the assistant.

>From OpenAI API docs: https://platform.openai.com/docs/guides/chat/introduction
>
>"gpt-3.5-turbo-0301 does not always pay strong attention to system messages. Future models will be trained to pay stronger attention to system messages."

>Tip 💡
>
>Tinker with the instructions in the prompt until you find the desired voice of your chatbot.

In [85]:
message_objects = []
message_objects.append({"role":"system", "content":"You're a chatbot helping customers with beauty-related questions and helping them with product recommendations"})


In [86]:
# Append the customer messagae
message_objects.append({"role":"user", "content": customer_input})

In [87]:
# Append prev relevant purchase
message_objects.append({"role":"user", "content": f"Please give me a detailed explanation of your recommendations"})
message_objects.append({"role":"user", "content": "Please be friendly and talk to me like a person, don't just give me a list of recommendations"})


In [88]:
# Create list of 3 products to recommend
products_list = []

for index, row in top_3_products_df.iterrows():
    brand_dict = {'role': "assistant", "content": f"{row['combined']}"}
    products_list.append(brand_dict)
products_list

[{'role': 'assistant', 'content': 'CeraVe, moisturizer, for dry skin'},
 {'role': 'assistant', 'content': 'Aveeno, moisturizer, for dry skin'},
 {'role': 'assistant',
  'content': 'First Aid Beauty, moisturizer, for dry skin'}]

In [89]:
# Append found products  
message_objects.append({"role": "assistant", "content": f"I found these 3 products I would recommend"})
message_objects.extend(products_list)
message_objects.append({"role": "assistant", "content":"Here's my summarized recommendation of products, and why it would suit you:"})
message_objects

[{'role': 'system',
  'content': "You're a chatbot helping customers with beauty-related questions and helping them with product recommendations"},
 {'role': 'user',
  'content': 'Hi! Can you recommend a good moisturizer for me?'},
 {'role': 'user',
  'content': 'Please give me a detailed explanation of your recommendations'},
 {'role': 'user',
  'content': "Please be friendly and talk to me like a person, don't just give me a list of recommendations"},
 {'role': 'assistant',
  'content': 'I found these 3 products I would recommend'},
 {'role': 'assistant', 'content': 'CeraVe, moisturizer, for dry skin'},
 {'role': 'assistant', 'content': 'Aveeno, moisturizer, for dry skin'},
 {'role': 'assistant',
  'content': 'First Aid Beauty, moisturizer, for dry skin'},
 {'role': 'assistant',
  'content': "Here's my summarized recommendation of products, and why it would suit you:"}]

In [80]:
message_objects

[{'role': 'system',
  'content': "You're a chatbot helping customers with beauty-related questions and helping them with product recommendations"},
 {'role': 'user',
  'content': 'Hi! Can you recommend a good moisturizer for me?'},
 {'role': 'user',
  'content': 'Please give me a detailed explanation of your recommendations'},
 {'role': 'user',
  'content': "Please be friendly and talk to me like a person, don't just give me a list of recommendations"},
 {'role': 'assistant',
  'content': 'I found these 3 products I would recommend'},
 {'role': 'assistant', 'content': 'Ole Henkrisen, moisturizer, for oily skin'},
 {'role': 'assistant', 'content': 'Aveeno, moisturizer, for dry skin'},
 {'role': 'assistant', 'content': 'CeraVe, moisturizer, for dry skin'},
 {'role': 'assistant',
  'content': "Here's my summarized recommendation of products, and why it would suit you:"},
 {'role': 'assistant',
  'content': "For oily skin types, I'd recommend Ole Henkrisen Facial Moisturizer. It's infused 

# 16. Call ChatGPT API

In [90]:
completion = openai.ChatCompletion.create(
  engine="GPT35",
  messages=message_objects
)
message_objects.append({"role": "assistant", "content": completion.choices[0].message['content']})
print(completion.choices[0].message['content'])

I would recommend the CeraVe Daily Moisturizing Lotion for dry skin because it contains ceramides and hyaluronic acid which help to retain moisture in the skin. It is also fragrance-free and non-comedogenic, making it suitable for sensitive skin. Additionally, it is easily absorbed and does not leave a greasy residue.


In [92]:
# question = "I have oily skin"
# message_objects.append({"role": "user", "content": question})
# response = openai.Embedding.create(
#     input=question,
#     engine="text-embedding-ada-002"
# )
# embeddings_customer_question = response['data'][0]['embedding']
# product_data_df['search_products'] = product_data_df.text_embedding.apply(lambda x: cosine_similarity(x, embeddings_customer_question))
# product_data_df = product_data_df.sort_values('search_products', ascending=False)
# top_3_products_df = product_data_df.head(3)
# products_list = []

# for index, row in top_3_products_df.iterrows():
#     brand_dict = {'role': "assistant", "content": f"{row['combined']}"}
#     products_list.append(brand_dict)
# print(products_list)
# message_objects.append({"role": "assistant", "content": f"I found these 3 products I would recommend"})
# message_objects.extend(products_list)
# message_objects.append({"role": "assistant", "content":"Here's my summarized recommendation of products, and why it would suit you:"})
# message_objects

completion = openai.ChatCompletion.create(
  engine="GPT35",
  messages=message_objects
)
message_objects.append({"role": "assistant", "content": completion.choices[0].message['content']})
print(completion.choices[0].message['content'])


For oily skin, I would recommend the Ole Henriksen Counter Balance Oil Control Hydrator. It is specifically formulated for oily skin and contains salicylic acid, niacinamide and glycerin which helps control oil production and hydrates without clogging pores. It also helps to reduce the appearance of pores, leaving your skin nourished and matte without feeling tight or dry. Additionally, it is vegan and cruelty-free.


In [93]:
message_objects


[{'role': 'system',
  'content': "You're a chatbot helping customers with beauty-related questions and helping them with product recommendations"},
 {'role': 'user',
  'content': 'Hi! Can you recommend a good moisturizer for me?'},
 {'role': 'user',
  'content': 'Please give me a detailed explanation of your recommendations'},
 {'role': 'user',
  'content': "Please be friendly and talk to me like a person, don't just give me a list of recommendations"},
 {'role': 'assistant',
  'content': 'I found these 3 products I would recommend'},
 {'role': 'assistant', 'content': 'CeraVe, moisturizer, for dry skin'},
 {'role': 'assistant', 'content': 'Aveeno, moisturizer, for dry skin'},
 {'role': 'assistant',
  'content': 'First Aid Beauty, moisturizer, for dry skin'},
 {'role': 'assistant',
  'content': "Here's my summarized recommendation of products, and why it would suit you:"},
 {'role': 'assistant',
  'content': 'I would recommend the CeraVe Daily Moisturizing Lotion for dry skin because it